In [6]:
# !export https_proxy=http://127.0.0.1:7890 http_proxy=http://127.0.0.1:7890 all_proxy=socks5://127.0.0.1:7890
# %pip install pandas
# %pip install pymysql

In [45]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
import pandas as pd
from datetime import datetime

import pymysql
pymysql.install_as_MySQLdb()

In [46]:
# 配置
HOSTNAME = "127.0.0.1"
PORT = "3306"
USERNAME = "root"
PASSWORD = "wangjin"
DATABASE = "notice"

In [47]:
app = Flask(__name__)
# 使用连接信息进行替换
app.config['SQLALCHEMY_DATABASE_URI'] = f"mysql+pymysql://{USERNAME}:{PASSWORD}@{HOSTNAME}:{PORT}/{DATABASE}?charset=utf8mb4"
db = SQLAlchemy(app)

In [48]:
import sqlalchemy
# 检查数据库是否链接成功
with app.app_context():
    with db.engine.connect() as conn:
        rs = conn.execute(sqlalchemy.text("select 1"))
        print(rs.fetchone())  # 链接成功，返回(1,)


(1,)


In [49]:
# NotificationModel定义
class NotificationModel(db.Model):
    __tablename__ = "notification_data"
    
    id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    title = db.Column(db.String(200), nullable=False)
    content = db.Column(db.Text, nullable=False)
    # 提取日期
    date = db.Column(db.DateTime, nullable=True)
    # 摘要
    summary = db.Column(db.Text, nullable=True)
    creator_id = db.Column(db.Integer)
    created_date = db.Column(db.DateTime, default=datetime.now)

with app.app_context():
    db.create_all()  # 创建表

In [50]:
# 导入表
def load_csv_into_db(filename):
    # 读取CSV
    df = pd.read_csv(filename)
    with app.app_context():
        # 遍历每一行并插入到数据库中
        for _, row in df.iterrows():
            # 解析日期
            date_str = row['日期']
            # 提取日期
            if isinstance(date_str, str):
                date = datetime.strptime(date_str, "%Y年%m月%d日") if "年" in date_str else pd.to_datetime(date_str)
            else:
                date = None  # 或者设置为其他默认值
            
            notification = NotificationModel(
                title=row['类别'],
                content=row['通知内容'],
                date=date,
                summary=row['概括内容'],
                creator_id=1
            )
            db.session.add(notification)
        db.session.commit()

# 加载CSV到数据库
load_csv_into_db("./dates_combined_data_summarized.csv")